Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [82]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [83]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [84]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head(3)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty


In [85]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head(3)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...


In [86]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head(3)

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty


In [87]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


## Задание 1.
Заменим html-сущности (к примеру: < > &). "<" заменим на “<” и "&" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

In [88]:
# импортируем модуль, изучить все его возможности для обработки текста
from html.parser import HTMLParser

In [89]:
h = HTMLParser()
combine_df.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."


В строке с индексом 49156 содержится escape-последовательность  &amp; - это будет контрольная строка, по которой мы проверим, произошло преобразование или нет

In [90]:
# с помощью лямбда-функции применяем преобразование к каждой строке и преобразованные значения помещаем в новый столбец
combine_df['unescaped'] = combine_df['tweet'].apply(lambda x: h.unescape(x))

In [91]:
# проверяем строку с индексом 49156: преобразование произведено успешно
combine_df.tail(5)

,id,label,tweet,unescaped
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads...","happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ...","my song ""so glad"" free download! #shoegaze ..."


## Задание 2
Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:
- для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
- для для замены @user на пробел, необходимо использовать re.sub() при применении функции необходимо использовать np.vectorize(function)

In [92]:
# создаём паттерн для поиска
user_pattern = re.compile("@[\w]*")

In [93]:
# создаём функцию для замены значений в ячейках
def replace_value(cell_value, pattern, new_value=''):
    new_text = re.sub(pattern, new_value, cell_value)
    return new_text

In [94]:
# создаём объект векторайзера
new_col = np.vectorize(replace_value)

In [95]:
# получаем вектор значений
text_list = combine_df['unescaped']
type(text_list)

pandas.core.series.Series

In [96]:
combine_df['user_drop'] = new_col(text_list, user_pattern)

In [97]:
combine_df.head()

,id,label,tweet,unescaped,user_drop
0,1,0.0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation


## Задание 3
Изменим регистр твитов на нижний с помощью .lower()

Применим преобразования с помощью векторайзера или функции лямбда

In [98]:
# создаём функцию, возвращающую текст в нижнем регистре
def set_lower(text):
    return text.lower()

In [99]:
# создаём объект векторайзера
lower_vr = np.vectorize(set_lower)

In [100]:
# вытаскиваем массив сообщений в вектор
text = combine_df['user_drop']

In [101]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['lower']=lower_vr(text)

In [102]:
combine_df.head(3)

,id,label,tweet,unescaped,user_drop,lower
0,1,0.0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...,thanks for #lyft credit i can't use cause th...
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty


In [103]:
# удалим "лишние столбцы"
combine_df.drop(['unescaped', 'user_drop'], axis=1, inplace=True)

## Задание 4
Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [104]:
# создаём функцию для замены сокращений: в ней мне удалось обойтись без использовани re,
# что, возможно, увеличит скорость работы
def replace_shortcuts(text, _dict):
    _dict_keys = list(_dict.keys())
    words = text.split()
    for word in words:
        real_word = ' '+word+' '
        if word in _dict_keys:
            new_word = ' '+_dict[word]+' '
            text = text.replace(real_word, new_word)
            #text = re.sub(real_word, new_word, text)
    return text

In [105]:
# создаём функцию для замены сокращений: в ней используется re модуль, возможно, это будет медленнее
def replace_shortcuts2(text, _dict):
    _dict_keys = list(_dict.keys())
    words = text.split()
    for word in words:
        real_word = ' '+word+' '
        if word in _dict_keys:
            new_word = ' '+_dict[word]+' '
            #text = text.replace(real_word, new_word)
            text = re.sub(real_word, new_word, text)
    return text

In [106]:
# сохраняем текст всех объектов в вектор
text = combine_df['lower']

In [107]:
# создаём объект векторайзера
vec_sh = np.vectorize(replace_shortcuts)

In [108]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['no_shortcuts']=vec_sh(text, apostrophe_dict)

In [109]:
combine_df.head(5)

,id,label,tweet,lower,no_shortcuts
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...,thanks for #lyft credit i cannot use cause t...
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation


## Задание 5
Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [110]:
# сохраняем текст всех объектов в вектор
text = combine_df['no_shortcuts']

In [111]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['no_shorts02']=vec_sh(text, short_word_dict)

In [112]:
combine_df.head()

,id,label,tweet,lower,no_shortcuts,no_shorts02
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...,thanks for #lyft credit i cannot use cause t...,thanks for #lyft credit i cannot use cause t...
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,#model i love you take with you all the time...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation


Проверим гипотезу о том, что использование re вместо стандартных методов-функций python увеличивает время

In [113]:
timeit(replace_shortcuts)

30.5 ns ± 3.05 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [114]:
timeit(replace_shortcuts2)

29.1 ns ± 4.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Наверно, эти результаты не очень релевантны и делать какие-то выводы на их основании не стоит

In [115]:
# удалим "лишние" столбцы
combine_df.drop(['lower','no_shortcuts'],axis=1, inplace=True)

In [116]:
# проверяем, всё ли нужное на месте
combine_df.head(4)

,id,label,tweet,no_shorts02
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i cannot use cause t...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love you take with you all the time...


## Задание 6
Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

Вероятно, функции, созданные мною на прошлых этапах, не подойдут для целей этой задачи. Ведь смайлики могут быть "прикреплены" к словам как "hello:)", в этом случае при разбивке по пробелам, они не станут "самстоятельным" (отдельным) словом

In [117]:
# создаём функцию для замены эмотиконов
def replace_emotics(text, _dict):
    _dict_keys = list(_dict.keys())
    for smailik in _dict_keys:
        new_word = ' '+_dict[smailik]+" "
        text = text.replace(smailik, new_word)
    return text

In [118]:
# сохраняем текст всех объектов в вектор
text = combine_df['no_shorts02']

In [119]:
# создаём объект векторайзера
vec_emo = np.vectorize(replace_emotics)

In [120]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['no_emotic']=vec_emo(text, emoticon_dict)

In [121]:
combine_df.head()

,id,label,tweet,no_shorts02,no_emotic
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i cannot use cause t...,thanks for #lyft credit i cannot use cause t...
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love you take with you all the time...,#model i love you take with you all the time...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation


## Задание 7
Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [122]:
def replace_value(cell_value, pattern,new_value = ''):
    new_text = re.sub(pattern, new_value, cell_value)
    return new_text

In [123]:
# воспользуемся ранее созданной функцией replace_value
pattern = r'[^\w\s]'
new_value = ' '

In [124]:
# сохраняем текст всех объектов в вектор
text = combine_df['no_emotic']

In [125]:
# создаём объект векторайзера
vec_punct = np.vectorize(replace_value)

In [126]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['no_punct']=vec_punct(text, pattern, new_value)

In [127]:
combine_df.drop(['no_shorts02', 'no_emotic'], axis=1, inplace=True)

In [128]:
combine_df.head(5)

,id,label,tweet,no_punct
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i cannot use cause t...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,model i love you take with you all the time...
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation


## Задание 8
Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [129]:
# воспользуемся ранее созданной функцией replace_value
pattern =  r'[^a-zA-Z0-9]'
new_value = ' '

In [130]:
# сохраняем текст всех объектов в вектор
text = combine_df['no_punct']

In [131]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['no_symb']=vec_punct(text, pattern, new_value)

In [132]:
combine_df.head(2)

,id,label,tweet,no_punct,no_symb
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i cannot use cause t...,thanks for lyft credit i cannot use cause t...


## Задание 9
Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [133]:
# воспользуемся ранее созданной функцией replace_value
pattern =  r'[^a-zA-Z]'
new_value = ' '

In [134]:
# сохраняем текст всех объектов в вектор
text = combine_df['no_symb']

In [135]:
# во вновь созданный столбец помещаем преобразованный текст
combine_df['no_numbers']=vec_punct(text, pattern, new_value)

In [136]:
combine_df.iloc[49153:49156,:]

,id,label,tweet,no_punct,no_symb,no_numbers
49153,49154,NaN,2_damn_tuff-ruff_muff__techno_city-(ng005)-web...,2_damn_tuff ruff_muff__techno_city ng005 web...,2 damn tuff ruff muff techno city ng005 web...,damn tuff ruff muff techno city ng web...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory left right polarisation tru...,thought factory left right polarisation tru...,thought factory left right polarisation tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid ð hairflip neverre...,feeling like a mermaid hairflip neverre...,feeling like a mermaid hairflip neverre...


## Задание 10
Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [137]:
def replace_one_letter(text):
    new_text = ' '.join([w for w in text.split() if len(w)>1])
    return new_text

In [138]:
vec_one_let = np.vectorize(replace_one_letter)

In [139]:
text = combine_df['no_numbers']

In [140]:
combine_df['no_one_letter']=vec_one_let(text)

In [141]:
combine_df.iloc[49153:49156,:]

,id,label,tweet,no_punct,no_symb,no_numbers,no_one_letter
49153,49154,NaN,2_damn_tuff-ruff_muff__techno_city-(ng005)-web...,2_damn_tuff ruff_muff__techno_city ng005 web...,2 damn tuff ruff muff techno city ng005 web...,damn tuff ruff muff techno city ng web...,damn tuff ruff muff techno city ng web ukhx in...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory left right polarisation tru...,thought factory left right polarisation tru...,thought factory left right polarisation tru...,thought factory left right polarisation trump ...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid ð hairflip neverre...,feeling like a mermaid hairflip neverre...,feeling like a mermaid hairflip neverre...,feeling like mermaid hairflip neverready forma...


In [142]:
combine_df.drop(['no_punct','no_symb','no_numbers'], axis=1, inplace=True)

## Задание 11
Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [143]:
combine_df.head(5)

,id,label,tweet,no_one_letter
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation


In [144]:
from nltk.tokenize import word_tokenize

In [145]:
te = 'when father is dysfunctional and is so selfish'

In [146]:
combine_df['tweet_token']=combine_df['no_one_letter'].apply(lambda x: nltk.word_tokenize(x))

In [147]:
combine_df.head()

,id,label,tweet,no_one_letter,tweet_token
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"


## Задание 12
Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [148]:
from nltk.corpus import stopwords

In [149]:
stop_words = stopwords.words("english")
print(type(stop_words),stop_words)

<class 'list'> ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 

In [150]:
%%time
combine_df['tweet_token_filtered']=combine_df['tweet_token'].apply(lambda x: [word for word in x if not word in stop_words])

CPU times: user 1.32 s, sys: 23.2 ms, total: 1.35 s
Wall time: 1.43 s


In [151]:
combine_df.head()

,id,label,tweet,no_one_letter,tweet_token,tweet_token_filtered
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"


## Задание 13
Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга

In [152]:
from nltk.stem import PorterStemmer

In [153]:
stemmer = PorterStemmer()

In [154]:
%%time
combine_df['tweet_stemmed']=combine_df['tweet_token_filtered'].apply(lambda x:[stemmer.stem(word) for word in x])

CPU times: user 11 s, sys: 164 ms, total: 11.1 s
Wall time: 12.4 s


In [155]:
combine_df.head()

,id,label,tweet,no_one_letter,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"


## Задание 14
Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [156]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [157]:
lemmer = WordNetLemmatizer()

In [158]:
%%time
combine_df['tweet_lemmatized']=combine_df['tweet_token_filtered'].apply(lambda x:[lemmer.lemmatize(word, wordnet.VERB) for word in x])

CPU times: user 2.04 s, sys: 70.2 ms, total: 2.11 s
Wall time: 2.21 s


In [159]:
combine_df.head()

,id,label,tweet,no_one_letter,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [160]:
combine_df = combine_df[['id','label','tweet','tweet_token', 'tweet_stemmed','tweet_lemmatized']]

In [161]:
combine_df.head()

,id,label,tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,"[thanks, for, lyft, credit, can, not, use, cau...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,"[factsguide, society, now, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


## Задание 15
Сохраним результат предобработки в pickle-файл.

In [162]:
combine_df.to_pickle("tweets.pkl")